# Data Preparation

"Predicting Vehicle Prices" merupakan sebuah perangkat analisis data yang bertujuan untuk memprediksi harga kendaraan berdasarkan profil kendaraan yang telah tersedia.

Kumpulan data ini berisi rincian kendaraan seperti merek, tahun produksi, jarak tempuh, jenis bahan bakar, tipe seller, transmisi, jumlah kepemilikan mobil sebelumnya. Variabel target dalam kasus ini adalah harga kendaraan, yang merupakan variabel numerik yang mencerminkan nilai kendaraan tersebut.

Variabel-variabel fitur ini akan diproses dan digunakan untuk membangun model machine learning regresi. Model ini akan digunakan untuk memprediksi harga kendaraan berdasarkan karakteristik yang dimiliki oleh setiap kendaraan dalam dataset.

Dalam hal ini, kita akan berfokus pada pengukuran metrik bisnis yang sesuai dengan permasalahan ini. Salah satu metrik yang relevan adalah "Root Mean Squared Error (RMSE)" yang mengukur seberapa akurat model dalam memprediksi harga kendaraan. Selain itu, kita juga dapat menggunakan metrik bisnis lain seperti "Mean Absolute Error (MAE)" untuk memberikan gambaran yang lebih lengkap tentang seberapa baik model ini dalam memprediksi harga kendaraan.

Dengan menggunakan metrik bisnis ini, kita dapat mengukur seberapa baik model ini dalam melakukan prediksi harga kendaraan, yang dapat sangat bermanfaat dalam industri penjualan kendaraan seperti otomotif. Semakin rendah nilai RMSE atau MAE, semakin baik model ini dalam memprediksi harga kendaraan, yang dapat membantu perusahaan atau individu yang terlibat dalam penjualan kendaraan untuk membuat keputusan yang lebih baik.

# Importasi Library

Melakukan importasi library yang dibutuhkan saat proses data preparation

In [59]:
#Import library untuk data preparation dan visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# import warnings for ignore the warnings
import warnings 
warnings.filterwarnings("ignore")

# import pickle and json file for columns and model file
import pickle
import json
import joblib
import copy

# Import train test split untuk splitting data
from sklearn.model_selection import train_test_split
import yaml
import src.util as util
from tqdm import tqdm
import os

In [60]:
os.getcwd()

'C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction'

In [61]:
params_dir = "config/config.yaml"

In [62]:
def load_params(param_dir):
    with open(param_dir, 'r') as file:
        params = yaml.safe_load(file)
        
    return params

In [63]:
params = load_params(params_dir)

In [64]:
params

{'raw_dataset_dir': 'dataset/1 - raw data/',
 'train_set_path': ['dataset/2 - processed/X_train.pkl',
  'dataset/2 - processed/y_train.pkl'],
 'valid_set_path': ['dataset/2 - processed/X_valid.pkl',
  'dataset/2 - processed/y_valid.pkl'],
 'test_set_path': ['dataset/2 - processed/X_test.pkl',
  'dataset/2 - processed/y_test.pkl'],
 'robust_scaler_train': ['dataset/3 - final/X_train_robust.pkl',
  'dataset/3 - final/y_train.pkl'],
 'robust_scaler_valid': ['dataset/3 - final/X_valid_robust.pkl',
  'dataset/3 - final/y_valid.pkl'],
 'robust_scaler_test': ['dataset/3 - final/X_test_robust.pkl',
  'dataset/3 - final/y_test.pkl'],
 'production_model_path': 'model/5 - Model Final',
 'model_robust_scaler': 'model/5 - Model Final/robust_scaler.pkl',
 'robust_scaler': 'model/5 - Model Final/',
 'model_final': 'model/5 - Model Final/xgboost_cv_robust.pkl',
 'print_debug': True,
 'int64_columns': ['Year', 'Kms_Driven', 'Owner'],
 'float64_columns': ['Present_Price', 'Selling_Price'],
 'object_colu

In [65]:
config_data = util.load_config()

# Read Data - Data Collection

Predicting Vehicle Prices saat ini menggunakan data yang disadur dari kaggle dengan laman :
 * https://www.kaggle.com/code/anandtalware/vehicle-sale-price-prediction-9-sept-2023/input?select=car+data.csv

In [66]:
def read_raw_data(config: dict) -> pd.DataFrame:
    # Create variable to store raw dataset
    raw_dataset = pd.DataFrame()

    # Raw dataset dir
    raw_dataset_dir = config["raw_dataset_dir"]

    # Look and load add CSV files
    for i in tqdm(os.listdir(raw_dataset_dir)):
        if i.endswith(".csv"):  # Hanya membaca file CSV
            raw_dataset = pd.concat([pd.read_csv(os.path.join(raw_dataset_dir, i)), raw_dataset])
            
    # Return raw dataset
    return raw_dataset

In [67]:
#Read Vehicle Prices dengan pandas
vehicles_data = read_raw_data(config_data)
vehicles_data.head()

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 208.47it/s]


,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [68]:
vehicles_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


In [69]:
# melihat unique value pada kolom kategori

# Menggunakan loop untuk melihat unique value pada masing-masing kolom
for column in vehicles_data.columns:
    if vehicles_data[column].dtype == 'object':
        unique_values = vehicles_data[column].unique()
        print(f"Unique values for column {column}:")
        print(unique_values)
        print('\n')

Unique values for column Car_Name:
['ritz' 'sx4' 'ciaz' 'wagon r' 'swift' 'vitara brezza' 's cross'
 'alto 800' 'ertiga' 'dzire' 'alto k10' 'ignis' '800' 'baleno' 'omni'
 'fortuner' 'innova' 'corolla altis' 'etios cross' 'etios g' 'etios liva'
 'corolla' 'etios gd' 'camry' 'land cruiser' 'Royal Enfield Thunder 500'
 'UM Renegade Mojave' 'KTM RC200' 'Bajaj Dominar 400'
 'Royal Enfield Classic 350' 'KTM RC390' 'Hyosung GT250R'
 'Royal Enfield Thunder 350' 'KTM 390 Duke ' 'Mahindra Mojo XT300'
 'Bajaj Pulsar RS200' 'Royal Enfield Bullet 350'
 'Royal Enfield Classic 500' 'Bajaj Avenger 220' 'Bajaj Avenger 150'
 'Honda CB Hornet 160R' 'Yamaha FZ S V 2.0' 'Yamaha FZ 16'
 'TVS Apache RTR 160' 'Bajaj Pulsar 150' 'Honda CBR 150' 'Hero Extreme'
 'Bajaj Avenger 220 dtsi' 'Bajaj Avenger 150 street' 'Yamaha FZ  v 2.0'
 'Bajaj Pulsar  NS 200' 'Bajaj Pulsar 220 F' 'TVS Apache RTR 180'
 'Hero Passion X pro' 'Bajaj Pulsar NS 200' 'Yamaha Fazer '
 'Honda Activa 4G' 'TVS Sport ' 'Honda Dream Yuga '
 'Baj

In [70]:
vehicles_data["Owner"].unique()

array([0, 1, 3], dtype=int64)

In [71]:
sorted(vehicles_data["Year"].unique())

[2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018]

In [72]:
vehicles_data["Car_Name"].nunique()

98

In [73]:
print(vehicles_data["Selling_Price"].min())
print(vehicles_data["Selling_Price"].max())

0.1
35.0


In [74]:
print(vehicles_data["Present_Price"].min())
print(vehicles_data["Present_Price"].max())

0.32
92.6


In [75]:
print(vehicles_data["Kms_Driven"].min())
print(vehicles_data["Kms_Driven"].max())

500
500000


Menghapus kolom yang tidak diperlukan dalam proses analisa

In [76]:
vehicles_data = vehicles_data.drop('Car_Name', axis=1)  # axis=1 berarti menghapus kolom, axis=0 akan menghapus baris

Sanity check

In [77]:
vehicles_data

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0
...,...,...,...,...,...,...,...,...
296,2016,9.50,11.60,33988,Diesel,Dealer,Manual,0
297,2015,4.00,5.90,60000,Petrol,Dealer,Manual,0
298,2009,3.35,11.00,87934,Petrol,Dealer,Manual,0
299,2017,11.50,12.50,9000,Diesel,Dealer,Manual,0


# Data Definition

Data yang digunakan merupakan data yang menggambarkan karakteristik vehicle berdasarkan harga jualnya. adapun fitur karakteristik vehicle yang digunakan antara lain

Data berisi 301 baris dan 8 kolom yaitu :

* Year : Tahun saat mobil tersebut dibeli
* Selling_Price : Harga saat mobil tersebut dijual
* Present_Price : harga mobil bekas showroom saat ini
* Kms_driven : Jarak tempuh yang sudah dilalui mobil tersebut
* Fuel_Type : Jenis bahan bakar mobil 
* Seller_Type : Tipe Penjual, perorangan atau dealer
* Transmission : Jenis Transmisi oper gigi mobil, manual atau otomatis
* Owner : Jumlah pemilik mobil sebelumnya.

# Data Validation

### Data Type

Mengecek tipe data apakah sudah sesuai dengan fitur dalam proses analisa

In [78]:
# Cek tipe data dengan menggunakan info()
vehicles_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           301 non-null    int64  
 1   Selling_Price  301 non-null    float64
 2   Present_Price  301 non-null    float64
 3   Kms_Driven     301 non-null    int64  
 4   Fuel_Type      301 non-null    object 
 5   Seller_Type    301 non-null    object 
 6   Transmission   301 non-null    object 
 7   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 18.9+ KB


### Check Nilai Null

Mengecek apakah data terdapat nilai null pada setiap kolom

In [79]:
#Menghitung nilai null pada kolom
vehicles_data.isnull().sum()

Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64

### Melihat statistik deskriptif dan Range Data Numerikal

Cek statistika deskriptif

In [80]:
#Melihat statistika deskriptif dengan describe()
vehicles_data.describe()

,Year,Selling_Price,Present_Price,Kms_Driven,Owner
count,301.000000,301.000000,301.000000,301.000000,301.000000
mean,2013.627907,4.661296,7.628472,36947.205980,0.043189
std,2.891554,5.082812,8.644115,38886.883882,0.247915
min,2003.000000,0.100000,0.320000,500.000000,0.000000
25%,2012.000000,0.900000,1.200000,15000.000000,0.000000
50%,2014.000000,3.600000,6.400000,32000.000000,0.000000
75%,2016.000000,6.000000,9.900000,48767.000000,0.000000
max,2018.000000,35.000000,92.600000,500000.000000,3.000000


### Melihat Dimensi Data

In [81]:
vehicles_data.shape

(301, 8)

# Data Splitting

Melakukan spliting data untuk memisahkan set training, set validation dan set test yang terbagi atas variabel x dan y

In [84]:
# Pemisahan Variabel X dan Y
X = vehicles_data.drop(columns = "Selling_Price")
y = vehicles_data["Selling_Price"]

In [85]:
#Split Data 80% training 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.25,
                                                    random_state = 123)

In [86]:
# Split data train menjadi train dan validation set
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, 
                                                    test_size=0.4, 
                                                    random_state=123)

In [87]:
#Menggabungkan x train dan y train untuk keperluan EDA
train_set = pd.merge(X_train, y_train, left_index=True, right_index=True)
#Sanity Check Data
train_set

,Year,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,Selling_Price
5,2018,9.83,2071,Diesel,Dealer,Manual,0,9.25
258,2015,13.60,25000,Petrol,Dealer,Manual,0,8.40
278,2016,8.40,4000,Petrol,Dealer,Manual,0,6.00
260,2016,13.60,29223,Petrol,Dealer,Manual,0,9.15
7,2015,8.61,33429,Diesel,Dealer,Manual,0,6.50
...,...,...,...,...,...,...,...,...
106,2014,3.45,16500,Petrol,Individual,Manual,1,1.35
83,2015,13.46,38000,Diesel,Dealer,Manual,0,12.50
17,2016,10.79,43000,Diesel,Dealer,Manual,0,7.75
230,2013,9.40,45000,Diesel,Dealer,Manual,0,6.15


## Final Result - Data Preparation

Ekspor Hasil data preparation dengan file pickle

In [88]:
util.pickle_dump(X_train, config_data["train_set_path"][0])
util.pickle_dump(y_train, config_data["train_set_path"][1])

util.pickle_dump(X_valid, config_data["valid_set_path"][0])
util.pickle_dump(y_valid, config_data["valid_set_path"][1])

util.pickle_dump(X_test, config_data["test_set_path"][0])
util.pickle_dump(y_test, config_data["test_set_path"][1])

In [25]:
joblib.dump(X_train, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\X_train.pkl")
joblib.dump(y_train, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\y_train.pkl")
joblib.dump(X_valid, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\X_valid.pkl")
joblib.dump(y_valid, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\y_valid.pkl")
joblib.dump(X_test, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\X_test.pkl")
joblib.dump(y_test, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\y_test.pkl")
joblib.dump(train_set, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\train_set.pkl") # Untuk Keperluan EDA

['C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\train_set.pkl']